In [1]:
import optax
import equinox as eqx
from jax import numpy as jnp, random as jr, vmap, nn

from pets.models import Ensemble
from pets.training import Buffer, train
from pets.envs.cartpole import CartPoleEnv

In [2]:
def collect_episode(env, buffer):
    (state, _), done = env.reset(), False
    while not done:
        action = env.action_space.sample()
        next_state, reward, done, truncated, info = env.step(action)
        buffer.add(state, action, reward, next_state)
        state = next_state
    return buffer

In [3]:
key = jr.PRNGKey(0)
env = CartPoleEnv()

num_epochs, num_train_epochs = 100, 10
batch_dim, hidden_dim, ensemble_dim = 32, 200, 5
state_dim, action_dim = env.observation_space.shape[0], env.action_space.shape[0]

buffer = Buffer(state_dim, action_dim, ensemble_dim)

for _ in range(num_epochs):
    buffer = collect_episode(env, buffer)

key, model_key = jr.split(key)

input_dim, output_dim = state_dim + action_dim, state_dim
model = Ensemble(input_dim, output_dim, hidden_dim, ensemble_dim, key=model_key)

optim = optax.adamw(learning_rate=1e-4, weight_decay=1e-5, eps=1e-8)
optim_state = optim.init(eqx.filter(model, eqx.is_inexact_array))

model, optim_state, losses = train(model, optim, optim_state, buffer, batch_dim, num_train_epochs)
print(losses)

(32, 5, 4) (32, 5, 4) (32, 5, 4)
()


IndexError: tuple index out of range